# Automated pipeline to run BFE corrections in stage 1
Takes an _uncal_ file and can apply either the 'grid' correction (directly applying ramp solutions from the closest pointing), or the 'interpolated' correction (gets ramp solutions based on an interpolated function using the offset from PSF centre). Requires a _rate_ file to centroid on. It is advised to run the 'grid' option on point source data with TA in one of the nominal point source dither positions, and 'interp' on point source data in a different position. Currently only applicable to channels 1 and 2.

In [ ]:
crds_cache_path = '' ## Set path to cache of JWST pipeline

In [ ]:
from jwst.pipeline import Detector1Pipeline

from astropy.io import fits

import core.linearity_coeff_ref_files as lin_ref

os.environ['CRDS_CONTEXT'] = '' #Set pmap
os.environ['CRDS_PATH'] = crds_cache_path #set cache path (defined in cell above)
os.environ['CRDS_SERVER_URL'] = 'https://jwst-crds.stsci.edu/'

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
inputDir = [''] #list of input directories (one per file)

rateDir = '' #location of previously reduced rate files for centroiding

outDir = '' #location to put output

refDir = './references/DISTORTION/' #location of distortion files (do NOT change unless location was manually changed)

file_list = [''] #list of files to reduce (should be uncal)


linFile = 'grid' #'grid' or 'interp'
saveRamp = False #Set to true if you wish to check the linearity of the reduced ramps
distVers = 'flt8'

In [ ]:
def run_pipeline(list_of_files, input_dir, output_dir, rate_dir, lin_file, step=1, ramp_save = False):

    for i in range(len(list_of_files)):
        
        file=list_of_files[i]
        print(list_of_files[i])
        
        if step==1:
            
            filesplit = file.split('_')
            ratefile = filesplit[0] + '_' + filesplit[1] + '_' + filesplit[2] + '_' + filesplit[3] + '_rate.fits'
            num = int((filesplit[2])[-1])
            
            rate = fits.open(rate_dir+ratefile)
            
            if lin_file == 'interp':
                lin_ref.gen_custom_linearity(rate,refDir,crds_cache_path,num=num,file_ver='v2',dist_ver=distVers)
                print('file generated')
                linearity_file = './references/LINEARITY/custom_linearity_ref_{}_v2.fits'.format(num)
            elif lin_file == 'grid':
                linearity_file = lin_ref.find_nearest_grid(rate,refDir,'./references/LINEARITY/',num=num,dist_ver=distVers)
                linearity_file = './references/LINEARITY/'+linearity_file
            
            rate.close()
            
            pipe = Detector1Pipeline()

            pipe.save_calibrated_ramp = ramp_save

            pipe.linearity.skip = False
            pipe.group_scale.skip = True
            pipe.dq_init.skip = False
            pipe.superbias.skip = False
            pipe.gain_scale.skip = True
#             pipe.emicorr.skip = False
            pipe.saturation.skip = False
            pipe.ipc.skip = True
            pipe.firstframe.skip = False
            pipe.lastframe.skip = False
            pipe.reset.skip = False
            pipe.saturation.n_pix_grow_sat = 0
            pipe.charge_migration.skip = True
            pipe.linearity.override_linearity = linearity_file
            pipe.rscd.skip = False
            pipe.dark_current.skip = False
            pipe.refpix.skip = False

            pipe.save_results = True
            pipe.output_dir = output_dir

            result = pipe.run(input_dir[i]+file)

In [ ]:
run_pipeline(file_list, inputDir, outDir, rateDir, lin_file = linFile, ramp_save = saveRamp)